These are our imports

In [ ]:
#imports 2021.2.576440691
import numpy as np
print("numpy imported")
import json
print("json imported")
import time
print("time imported")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
print("tensorflow imported") 
from keras import Sequential
print("keras imported")
import config
channelID,serverID,targetID = config.getScope()
print("config imported")
import pickle
print("pickle imported")
import random
print("random imported")
import string
print("string imported")
from scipy.sparse import csc_matrix
print("scipy imported")

Run the section below to process your data

In [ ]:
def exists(filename):
    try:
        f = open(filename, "rb")
        f.close()
        return True
    except:
        return False
if not exists("message.list"):
    channelID,serverID,targetID = config.getScope()
    daTestDoc = open("TestDoc","r",encoding="utf-8")
    daMessageList = eval(daTestDoc.read())
    with open('message.list','wb') as messageList_file:
        pickle.dump(daMessageList,messageList_file)
        print(":flushed:")
print('not flushed')

In [ ]:
inputList,outputList = [],[]
with open("message.list",'rb') as daMessageList:
    channelID,serverID,targetID = config.getScope()
    daMessageList = pickle.load(daMessageList)
    for i in range(len(daMessageList)):
        if i != len(daMessageList)+1:
            message = daMessageList[i]
            if message["author"]["id"] == str(targetID):
                #if i != 0:
                if True:
                    previousMessage = daMessageList[i-1]
                    if previousMessage["author"]["id"] == str(targetID):
                        if len(outputList) != 0:    
                            outputList[len(outputList)-1] += "   " + message["content"]
                    else:
                        if type(message.get("referenced_message"))!=dict:
                            outputList.append(message["content"])
                            message = daMessageList[i+1]
                            inputList.append(message["content"])
                        else:
                            inputList.append(message["referenced_message"]["content"])
                            outputList.append(message["content"])

        myIndice = random.randint(0,len(inputList))

    # for message in range(len(inputList)):
    # 	inputList[message] = removePunctuation(removeCaps(inputList[message]))
    with open('input.list','wb') as inputList_file:
        pickle.dump(inputList,inputList_file)
    with open('output.list','wb') as outputList_file:
        pickle.dump(outputList,outputList_file)
    print("Complete! (1/2)")
print(inputList[random.randint(0,1000)])

with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        
        removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
        removeCaps = lambda x: x.lower()

        for message in range(len(inputList)):
            inputList[message] = removePunctuation(removeCaps(inputList[message]))
            outputList[message] = removePunctuation(removeCaps(outputList[message]))
            inputList[message] = "<bos> " + inputList[message] + " <eos>"
            outputList[message] = "<bos> " + outputList[message] + " <eos>"
        #inputList = np.array(inputList)
        #outputList = np.array(outputList)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)




daShape = None


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        vocabList = {}
        vocabDict = []#i am good at naming things C:
        for x in inputList:
            wordList = x.split()
            for word in wordList:
                if word in vocabList:
                    vocabList[word] += 1
                else:
                    vocabList[word] = 1
        for x in outputList:
            wordList = x.split()
            for word in wordList:
                if word in vocabList:
                    vocabList[word] += 1
                else:
                    vocabList[word] = 1
        for word in vocabList.keys():
            if vocabList[word] > 2:
                vocabDict.append(word)
        tokenizer = Tokenizer(oov_token=2)
        #print(type(inputList+outputList))
        tokenizer.fit_on_texts(vocabDict)
        #print(inputList[0:10])
        inputList = tokenizer.texts_to_sequences(inputList)
        outputList = tokenizer.texts_to_sequences(outputList)

        #print(inputList)
        
        inputList = pad_sequences(inputList,padding='post')
        outputList = pad_sequences(outputList,padding='post')


        print(inputList.shape)
        print(outputList.shape)
        if inputList.shape[1] > outputList.shape[1]:
            daMaxLen = inputList.shape[1]
        else:
            daMaxLen = outputList.shape[1]
        print(daMaxLen)


        print(inputList[0])
        inputList = pad_sequences(inputList, maxlen = daMaxLen,padding='post')
        outputList = pad_sequences(outputList, maxlen = daMaxLen,padding='post')

        print(inputList[0])

        daShape = outputList.shape

        

        inputList = np.expand_dims(inputList,axis=0)
        outputList = np.expand_dims(outputList,axis=0)

        #outputList = to_categorical(outputList)


        #print(inputList[0])
        #print(inputList.shape)
        inputList,outputList = inputList[0],outputList[0]
        #print(inputList.shape)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)
        print("Complete!")
print("sus")

In [ ]:
with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
l = tokenizer.word_index

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding,LSTM
x = 8

eInput = Input(shape = (daMaxLen,),name="encoder_input")
vocabSize = len(tokenizer.word_index) + 1 #gets the vocab size from tokenizer, duh

print("maxLen is " + str(daMaxLen))

EmbeddingLayer = Embedding(vocabSize,output_dim=x,input_length=daMaxLen,trainable=True,name="Embedding")#vocabsize, number of output dimensions(try 8), inputlength(the padded length of the things)

eEmbed = EmbeddingLayer(eInput)


EncoderLSTM = LSTM(64,return_sequences=True,return_state=True,name="Encoder_LSTM")#number of neurons, input shape arg1 is number of timesteps, arg2 is dimensions of words/embeddings
_,encoder_h,encoder_c = EncoderLSTM(eEmbed)

dInput = Input(shape = (daMaxLen,),name="decoder_input")

dEmbed = EmbeddingLayer(dInput)

print(dEmbed.shape)

DecoderLSTM = LSTM(64,return_sequences=True,return_state=True,name="Decoder_LSTM")
decoder_output, _, _ = DecoderLSTM(dEmbed,initial_state=[encoder_h,encoder_c])
print(decoder_output.shape)

dense = Dense(vocabSize,activation = 'softmax',name="Dense")
print(vocabSize)
finalOutput = dense(decoder_output)
print(finalOutput.shape)
#Decode_Embedding = Dense(daMaxLen,activation='softmax')
#finalOutput = Decode_Embedding(decoder_output)



#Decode_Embedding2 = Dense(vocabSize,input_dim=dense_output.shape, activation="softmax")
#finalOutput = Decode_Embedding2(dense_output)

In [ ]:
This is an error so that I can hit run all and Jupyter won't run the next cell :)

In [ ]:
print(len(vocabList))
daModel.summary()
plot_model(daModel)

In [ ]:
from keras.callbacks import CSVLogger
# #Inverse tokenization
# reverse_word_map = tokenizer.word_index
# #print(reverse_word_map.keys())
# reverse_word_map = {v: k for k, v in reverse_word_map.items()}
# reverse_word_map[0] = ""

#Model


                        #inputs       #outputs
training_model = Model([eInput,dInput], finalOutput)#Compiling
training_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])#Training
print(training_model.summary())

csv_logger = CSVLogger("model16_history_lconsole.logog.csv", append=True)


checkpoint = [csv_logger]
checkpoint = [ModelCheckpoint(filepath='model1'),csv_logger]

def generator(features,labels,batch_size):#this is necessary to avoid memory overload
    batch_features = [np.zeros((batch_size,daMaxLen)),np.zeros((batch_size,daMaxLen))]
    batch_labels = np.zeros((batch_size,daMaxLen,vocabSize))
    while True:
        for i in range(batch_size):
            index = random.randint(0,len(labels)-1)
            batch_features[0][i] = features[index]
            batch_features[1][i] = features[index]
            labelsPointer = labels[index]
            # labelsPointer = batch_features[1][i][1:]
            # labelsPointer = np.concatenate((labelsPointer,np.zeros((1,))),axis=0)
            batch_labels[i] = to_categorical(labelsPointer,vocabSize)
        yield batch_features,batch_labels

#maybe have to make input/output into nparray
epochs,batch_size = 10000,64
training_model.fit(generator(inputList,outputList,batch_size), steps_per_epoch = len(inputList), epochs = epochs,callbacks=checkpoint,verbose = 1)

In [ ]:
import time
daModel = keras.models.load_model("model1")

#Embedding
EmbeddingLayer.set_weights(daModel.get_layer("Embedding").get_weights())
#EncoderLSTM
EncoderLSTM.set_weights(daModel.get_layer("Encoder_LSTM").get_weights())
#DecoderLSTM
DecoderLSTM.set_weights(daModel.get_layer("Decoder_LSTM").get_weights())
#Dense
dense.set_weights(daModel.get_layer("Dense").get_weights())

encoderStateModel = Model([eInput],[encoder_h,encoder_c])
decoder_input_h = Input(shape=(64,))
decoder_input_c = Input(shape=(64,))

dn = EmbeddingLayer.get_weights()[0]

decoder_outputs, decoder_h, decoder_c = DecoderLSTM(dEmbed, initial_state= [decoder_input_h,decoder_input_c])

decoderOutputModel = Model([dInput]+[decoder_input_h,decoder_input_c], [decoder_outputs]+[decoder_h,decoder_c])

modelInput = "<bos> have you done deca <eos>"

removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
removeCaps = lambda x: x.lower()


modelInput = removePunctuation(removeCaps(modelInput))
modelInput = tokenizer.texts_to_sequences([modelInput])
modelInput = pad_sequences(modelInput, maxlen = daMaxLen,padding='post')
modelInput = np.expand_dims(modelInput,axis=0)

#modelInput = inputList[1]
#modelInput = np.expand_dims(modelInput,axis=0)
#modelInput = np.expand_dims(modelInput,axis=0)


#outputList = to_categorical(outputList)
outputArray = np.zeros((1,1))
outputArray[0,0] = tokenizer.word_index["bos"]

#modelInput = modelInput.astype(np.uint8)

print(modelInput[0])
p = encoderStateModel.predict(modelInput[0])


inv_wordIndex = {v: k for k, v in tokenizer.word_index.items()}
inv_wordIndex[0] = "fil"
mostLikely = ""
outputString = ""
counter = 0
print(inv_wordIndex[1])

#returnString = ''
#for i in modelInput[0][0]:
#    returnString += inv_wordIndex[i] + ' '
#print(returnString)
#returnString2 = ''
#for i in outputList[2283]:
#    returnString2 += inv_wordIndex[i] + ' '
#print(returnString2)
#def beamSearch(k,):
while not mostLikely in ["eos"]:
    if counter == 20:
        break
    dOutput, d_h, d_c = decoderOutputModel.predict([outputArray]+p)
    wordProbabilities = dense(dOutput)
    print("sex\n")
    print(wordProbabilities[0][0])
    mostLikely = np.argmax(wordProbabilities[0][0])
    x = (np.argpartition(wordProbabilities[0][0], -4)[-4:])
    if [mostLikely] == outputArray and counter != 0:
        mostLikely = x[0]
    for y in x:
        print(str(inv_wordIndex[y]) + " - " + str(wordProbabilities[0][0][y]))
    outputArray = np.zeros((1, 1))
    outputArray[0, 0] = mostLikely
    print(outputArray)
    mostLikely = inv_wordIndex[mostLikely]
    print(x)
    
    outputString += " "+str(mostLikely)+" "
    p = [d_h,d_c]
    print(outputString)
    counter += 1


The tf.Tensor() output, a textual representation of an NP array containing the final output, contains a scalar probability for each possible next word.


In [ ]:
daModel = keras.models.load_model("model3")
print(daModel.history.keys())